<p><font size="6"><b>Spatial relationships and joins</b></font></p>


> *GCCA+ phase 2 - Geopyhton training*  
> *June, 2023*
>
> *© 2023, Jasper Feyen  (<mailto:jasperfeyen@hotmail.com>)*
---

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas

In [ ]:
countries = geopandas.read_file("data/ne_10m_admin_0_countries.zip")
cities = geopandas.read_file("data/ne_110m_populated_places.zip")
rivers = geopandas.read_file("data/ne_50m_rivers_lake_centerlines.zip")

## Spatial relationships

An important aspect of geospatial data is that we can look at *spatial relationships*: how two spatial objects relate to each other (whether they overlap, intersect, contain, .. one another).

The topological, set-theoretic relationships in GIS are typically based on the DE-9IM model. See https://en.wikipedia.org/wiki/Spatial_relation for more information.

![](../img/TopologicSpatialRelarions2.png)
(Image by [Krauss, CC BY-SA 3.0](https://en.wikipedia.org/wiki/Spatial_relation#/media/File:TopologicSpatialRelarions2.png))

### Relationships between individual objects

Let's first create some small toy spatial objects:

A polygon <small>(note: we use `.item()` here to to extract the scalar geometry object from the GeoSeries of length 1)</small>:

In [ ]:
countries[countries['name'] == 'Suriname']['geometry'].item()

In [ ]:
suriname = countries.loc[countries['name'] == 'Suriname', 'geometry'].item()

In [ ]:
suriname = countries.loc[countries['name'] == 'Suriname', 'geometry'].item()

Two points:

In [ ]:
cities.loc[cities['name'] == 'Paramaribo']

In [ ]:
cities[cities['name']== 'Paramaribo'].geometry.item()

In [ ]:
paramaribo = cities.loc[cities['name'] == 'Paramaribo','geometry'].item()

In [ ]:
georgetown = cities.loc[cities['name'] == 'Georgetown','geometry'].item()

In [ ]:
georgetown = cities.loc[cities['name'] == 'Georgetown', 'geometry'].item()

And a linestring:

In [ ]:
from shapely.geometry import LineString
line = LineString([paramaribo, georgetown])

In [ ]:
georgetown.distance(paramaribo)

Let's visualize those 4 geometry objects together (I only put them in a GeoSeries to easily display them together with the geopandas `.plot()` method):

In [ ]:
geopandas.GeoSeries([suriname, paramaribo, georgetown, line]).plot(cmap='tab10')

Paramaribo ligt uiteraard binnen Suriname. Dit is m.a.w. een spatiale relatie. Dit kunnen we ook gaan nagaan binnen geopandas:

In [ ]:
paramaribo

In [ ]:
paramaribo.within(suriname)

En omgekeerd bevat Suriname de stad Paramaribo

In [ ]:
suriname.contains(paramaribo)

Georgetown ligt natuurlijk buiten Suriname:

In [ ]:
suriname.contains(georgetown)

In [ ]:
georgetown.within(suriname)

De lijn tussen Paramaribo en Georgetown ligt gedeeltelijk binnen Suriname, maar niet volledig. Dit is een intersectie

In [ ]:
suriname.contains(line)

In [ ]:
line.intersects(suriname)

### Spatiale relaties en GeoDataFrames

Deze simpele methodes die we met `shapely` geometries hebben uitgevoerd kunnen ook toegepast worden op volledige `GeoSeries` / `GeoDataFrame` objecten.

Dit maakt het mogelijk om op een snelle manier dergelijke spatiale queries uit te voeren.

Bijvoorbeeld, als we op zoek zijn naar alle landen waar Paramaribo zich in bevindt, kunnen we dus de volledige `countries` database doorzoeken, op basis van de Geometrie via `contains` .


In [ ]:
countries

In [ ]:
countries.contains(suriname)

Het resultaat is een *Boolean* lijst, die weergeeft voor welke rijen in `countries` aan de spatiale relatie voldoet. We kunnen deze gebruiken om ook de landen met Paramaribo er uit te filteren.

In [ ]:
countries[countries.contains(suriname)]

En inderdaad, Suriname is het enige land ter wereld met een stad die Paramaribo heet. Hoe toevallig!

Verder kunnen we ook nagaan door welke landen een rivier stroomt. Hiervoor hebben we de rivieren-database ter beschikking

In [ ]:
amazon = rivers[rivers['name'] == 'Amazonas'].geometry.item()

In [ ]:
amazon

In [ ]:
countries.crosses(amazon)

In [ ]:
countries[countries.crosses(amazon)]  # of .intersects

<div class="alert alert-info" style="font-size:120%">

**REFERENTIE**:

Overzicht van de mogelijke functies om een spatiale relatie te bekijken (*spatial predicate functions*):

* `equals`
* `contains`
* `crosses`
* `disjoint`
* `intersects`
* `overlaps`
* `touches`
* `within`
* `covers`
* `covered_by`


Zie ook https://shapely.readthedocs.io/en/stable/manual.html#predicates-and-relationships voor een overzicht van elk van deze mogelijkheden

De wiki https://en.wikipedia.org/wiki/DE-9IM kan ook helpen bij de beschrijving van elk.

</div>

## OEFENINGEN!

We zullen opnieuw te werk gaan met onze Mangrove-datasets. We lezen ze in, maar gaan ze ook onmiddellijk herprojecteren naar dezelfde CRS

In [ ]:
districten = geopandas.read_file("data/Suriname_districts.geojson").to_crs(epsg=32621)
plotdata = geopandas.read_file("data/mangrove_2022.gpkg").to_crs(epsg=32621)

<div class="alert alert-success">

**OEFENING 1: Mangrove educatiecentrum**

Het Mangrove Educatiecentrum is een museum in Totness in het district Coronie in Suriname. Het educatiecentrum is een spinoff van een workshop over kustbescherming bij 's Lands Bosbeheer en werd opgezet om mensen bewuster te maken van het belang van de mangrovebossen.
    
De locatie van het educatiecentrum is: x = 573391.7 , y=650302.4

* Maak een Shapely point object aan met de coordinaten van het Educatiecentrum en maak er een variabele `educatiecentrum` van. Print het resultaat
* Bekijk of het educatiecentrum in het district Nickerie gelegen is (gegeven).
* Ga na in welk district het centrum ligt:
    * Maak een *boolean* mask (of filter) die weergeeft of het educatiecentrum wel (True) of niet (False) in elk district ligt
    * Filter `districten` op basis van dit boolean

<details><summary>Hints</summary>

* De `Point` klasse is beschikbaar in de `shapely.geometry` submodule
* Je kunt een punt toevoegen door een X en Y coordinaat in de `Point()` constructor te voeren.
* Middels `within()` kun je nagaan of een object zich binnen een 2 object bevindt (used as `geometry1.within(geometry2)`).
* Middels `contains()` kun je nagaan of een eerste geometry een 2e bevat (used as `geometry1.contains(geometry2)`).

</details>

</div>

In [ ]:
# Import the Point geometry
from shapely.geometry import Point

In [ ]:
# %load _solutions/05-spatial-relationships-1.py
# Punt toevoegen
educatiecentrum = Point(573391.7,650302.4)

In [ ]:
educatiecentrum

In [ ]:
# STAP 1 - geometry van Nickerie zoeken

nickerie = districten[districten['DISTR_NAAM'] == 'Nickerie'].geometry.item()

In [ ]:
educatiecentrum.within(nickerie)

In [ ]:
#STAP 2 -- Filteren van districten

districten[districten.contains(educatiecentrum)]

In [ ]:
# %load _solutions/05-spatial-relationships-2.py
# Nagaan of educatiecentrum binnen Nickerie ligt
educatiecentrum.within(district_nickerie)

In [ ]:
# Om Nickerie naar een polygoon te brengen (Polygon)
district_nickerie = districten.loc[districten['DISTR_NAAM']=='Nickerie', 'geometry'].item()
plot = plotdata.loc[plotdata['id_plot'] == '4_1', 'geometry'].item()

In [ ]:
# %load _solutions/05-spatial-relationships-3.py

<div class="alert alert-success">

**OEFENING 2 - AFSTAND TOT DE DICHTSTE MANGROVEPLOT**

Voor een excursie zijn geïnteresseerd in de mangrove inventarisplot dicht bij het educatiecentrum.
    
Om dit te bepalen kunnen we de (loodrechte) afstand van elke plot naar het educatiecentrum berekenen. Op basis van dit resultaat kunnen we een *mask* aanmaken die elke plot binnen een straal van 10km bevat. Hierbij krijgen we een `True` waarde als een plot binnen deze straal ligt en een `False` indien niet.

* Bereken de afstand tussen elke mangrove plot en het educatiecentrum. Ken dit toe aan de variabele `dist_centrum`.
* print de afstand tot het dichtstse station (wat is de minimum-waarde van `dist_centrum`?)
* Selecteer the rijen van de `plotdata` GeoDataFrame waar de afstand tot het educatiecentrum minder dan 10 km is (opgelet, de afstand is berekend in meter). Het resultaat noem je `plots_centrum`.

<details><summary>Hints</summary>  
* Om de afstand tussen 2 geometriën te bereknen maak je gebruik van `distance()` methode ( `geometry1.distance(geometry2)`).
* De `.distance()` methode of werkt ]element-wise]: het wordt uitgevoerd voor elke geometrie binnen de GeoDataFrame.
* Een Series heeft een `.min()` methode om het minimum te vinden
* Om aan de 10km conditie te voldoen kunnen we een conditional gebruikten, bijvoorbeeld `distance < 10000`.

</details>

</div>

In [ ]:
plotdata['dist_centrum'] = plotdata.distance(educatiecentrum)

In [ ]:
plotdata = plotdata.sort_values(by='dist_centrum')

In [ ]:
plots_centrum = plotdata[plotdata['dist_centrum'] < 10000]

In [ ]:
# %load _solutions/05-spatial-relationships-4.py
# Berekenen van afstand plots - mangrovecentrum
dist_centrum = plotdata.distance(educatiecentrum)

In [ ]:
# %load _solutions/05-spatial-relationships-5.py
# kortste afstand
dist_centrum.min()

In [ ]:
# %load _solutions/05-spatial-relationships-6.py

In [ ]:
# EXTRA:

m = plots_centrum.explore(marker_kwds=dict(radius=5))
geopandas.GeoSeries([educatiecentrum], crs='EPSG:32621').explore(m=m, color='red', marker_kwds=dict(radius=5))

---

## Spatial joins

In the previous section of this notebook, we could use the spatial relationship methods to check in which country a certain city was located. But what if we wanted to perform this same operation for every city and country? For example, we might want to know for each city in which country it is located.  

In tabular jargon, this would imply adding a column to our cities dataframe with the name of the country in which it is located. Since country name is contained in the countries dataset, we need to combine - or "join" - information from both datasets. Joining on location (rather than on a shared column) is called a "spatial join".

So here we will do:

- Based on the `countries` and `cities` dataframes, determine for each city the country in which it is located.
- To solve this problem, we will use the the concept of a "spatial join" operation: combining information of geospatial datasets based on their spatial relationship.

### Recap - joining dataframes

Pandas provides functionality to join or merge dataframes in different ways, see https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/ for an overview and https://pandas.pydata.org/pandas-docs/stable/merging.html for the full documentation.

To illustrate the concept of joining the information of two dataframes with pandas, let's take a small subset of our `cities` and `countries` datasets:

In [ ]:
cities2 = cities[cities['name'].isin(['Bern', 'Brussels', 'London', 'Paris'])].copy()
cities2['iso_a3'] = ['CHE', 'BEL', 'GBR', 'FRA']

In [ ]:
cities2

In [ ]:
countries2 = countries[['iso_a3', 'name', 'continent']]
countries2.head()

We added a 'iso_a3' column to the `cities` dataset, indicating a code of the country of the city. This country code is also present in the `countries` dataset, which allows us to merge those two dataframes based on the common column.

Joining the `cities` dataframe with `countries` will transfer extra information about the countries (the full name, the continent) to the `cities` dataframe, based on a common key:

In [ ]:
cities2.merge(countries2, on='iso_a3')

**But** for this illustrative example we added the common column manually, it is not present in the original dataset. However, we can still know how to join those two datasets based on their spatial coordinates.

### Recap - spatial relationships between objects

In the previous section, we have seen the notion of spatial relationships between geometry objects: within, contains, intersects, ...

In this case, we know that each of the cities is located *within* one of the countries, or the other way around that each country can *contain* multiple cities.

We can test such relationships using the methods we have seen in the previous notebook:

In [ ]:
france = countries.loc[countries['name'] == 'France', 'geometry'].squeeze()

In [ ]:
cities.within(france)

The above gives us a boolean series, indicating for each point in our `cities` dataframe whether it is located within the area of France or not.  
Because this is a boolean series as result, we can use it to filter the original dataframe to only show those cities that are actually within France:

In [ ]:
cities[cities.within(france)]

We could now repeat the above analysis for each of the countries, and add a column to the `cities` dataframe indicating this country. However, that would be tedious to do manually, and is also exactly what the spatial join operation provides us.

*(note: the above result is incorrect, but this is just because of the coarse-ness of the countries dataset)*

## Spatial join operation

<div class="alert alert-info" style="font-size:120%">

**SPATIAL JOIN** = het overbrengen van attributen van de ene laag naar de andere op basis van hun spatiale relatie

Verschillende onderdelen van deze operatie:

* De GeoDataFrame waaraan we informatie willen toevoegen.
* De GeoDataFrame die de informatie bevat die we willen toevoegen.
* De spatiale relatie ("predicate") die we willen gebruiken om beide datasets te matchen ('intersects', 'contains', 'within').
* Het type join: linkse of binnenste join.


![](../img/illustration-spatial-join.svg)

</div>

In dit geval willen we de `cities` met de informatie van de `countries` dataframe samenvoegen, gebaseerd op the spatiale relatie tussen beide datasets.

Hiervoor maken we gebruik van de  [`geopandas.sjoin`](http://geopandas.readthedocs.io/en/latest/reference/geopandas.sjoin.html) functie:

In [ ]:
geopandas.sjoin(cities,countries, predicate='within', how='left')

In [ ]:
joined = geopandas.sjoin(cities, countries, predicate='within', how='left')

In [ ]:
joined

In [ ]:
joined[joined["name_right"] == "Suriname"]

In [ ]:
joined['continent'].value_counts()

## Oefenen!

Andermaal maken we gebruik van de mangrove-dataset

In [ ]:
districten = geopandas.read_file("data/Suriname_districts.geojson").to_crs(epsg=32621)
plotdata = geopandas.read_file("data/mangrove_2022.gpkg").to_crs(epsg=32621)

<div class="alert alert-success">

**Oefening 4:**

* Bepaal voor elke mangrove plot in welke district het is gelegen. Noem het resultaat `joined`.

<details><summary>Hints</summary>

- De `geopandas.sjoin()` functie heeft 2 argumenten: 1e argument is de dataframe waar we informatie aan willen toevoegen. Het 2e argument waar we de info vandaan willen halen. 
</details>

</div>

In [ ]:
geopandas.sjoin(plotdata,districten, predicate = 'within', how='left' )

In [ ]:
joined = geopandas.sjoin(plotdata,districten,predicate = 'within', how = "left")

In [ ]:
joined

In [ ]:
joined['DISTR_NAAM'].value_counts()